In [ ]:
from aicsimageio import AICSImage
import os
from pydicom import dcmread
import imageio
import pandas as pd
import tifffile
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Load 3D Dicom Data and Save them as Tiff

#### Loading 2D dicom image files that are part of a 3D structure and save them as a 3D tiff-file
The dicom 2D files need to be loaded in the correct order (according to the attribute 'SliceLocation'), put into a 3D numpy array, and then saved as tiff. This tiff file can then be loaded into a napari viewer for annotation.

In [ ]:
#path_to_file = Path('/Users/stephanehess/Documents/CAS_AML/hundeknie_data/Segmentation_sheep_dog_Hess/dog_1_DIRU_2022_04_dog_03/PAT_0002/STD_0000/SER_0005/OBJ_0001')

### Define path to load file: 

In [ ]:
root_path = Path('/Users/stephanehess/Documents/CAS_AML/hundeknie_data/Segmentation_sheep_dog_Hess')

In [ ]:
root_path

In [ ]:
dog_3_path = root_path / 'dog_1_DIRU_2022_04_dog_03' 

In [ ]:
dog_4_path = root_path / 'dog_4'

In [ ]:
#path_to_file = dog_3_path / 'PAT_0002/STD_0000/SER_0005/OBJ_0001'
path_to_files = dog_4_path / 'NC/3DT2/OBJ_0001'

### Move to folder where the files are: 

In [ ]:
os.chdir(path_to_files)

### Load a file as an example:

In [ ]:
file = dcmread('IM_0360')

In [ ]:
image_data = file.PixelData
print(type(image_data))

The image_data object has an attribute 'pixel_array' that contains the actual data
as a numpy array:

In [ ]:
image_data = file.pixel_array
image_data

In [ ]:
image_data.shape

In [ ]:
type(image_data)

In [ ]:
os.getcwd()

### Get a list of all dicom files in the folder:

In [ ]:
files = os.listdir()
files[0:7]

The files are in random order, however, sorting by file name will not suffice.

In [ ]:
files.sort()

In [ ]:
files

### Select all files with a SliceLocation and extract data and SliceLocation:

In [ ]:
# List to put in the numpy array data from each file:
image_list = []
# List to put in the SliceLocation from each file: 
slice_num_list = []
# Counter to keep track of how many files were skipped 
# because they did not have a SliceLocation:
skipcount = 0
# Loop through all the files:
for file in files:
    #print(file)
    # Load the file:
    image_data = dcmread(file)
    # If the file has an attribute SliceLocation ...
    if hasattr(image_data, 'SliceLocation'):
        
        # Append the SliceLocation to the respective list: 
        slice_num_list.append(image_data.SliceLocation)
        # Append the image data to the respective list: 
        image_2d = image_data.pixel_array
        image_list.append(image_2d)
    # If no attribute SliceLocation count: 
    else:
        skipcount = skipcount + 1

Check how many slices did not have an attribute SliceLocation:

In [ ]:
skipcount

### Put slice location (SliceLocation) and the data in a dataframe and then sort the rows by slice numbers:

In [ ]:
image_slices = pd.DataFrame({'slice_number': slice_num_list, 'image_slice':image_list})
image_slices.head

In [ ]:
image_slices.sort_values(by='slice_number', axis=0, inplace=True)

### Check if rows are ordered by slice location:

In [ ]:
image_slices.head()

### Extract the data (image slices) and put them back into a numpy array:

In [ ]:
image_slices = list(image_slices.image_slice)

In [ ]:
image_3d = np.stack(image_slices, axis=-1)

In [ ]:
type(image_3d)

In [ ]:
image_3d.shape

### Loop through the slices and plot them to check if they are arranged according to a 3D structure that makes sense:

In [ ]:
l2 = list(np.arange(20, 480, 20))

In [ ]:
n_rows = 12
images_per_row = 2
fig, axs = plt.subplots(nrows=n_rows, ncols=images_per_row, figsize=(8, n_rows * 3))
axs = axs.flatten()
#for index_count, index_image in enumerate(range(330, 380)):
for index_count, index_image in enumerate(l2):
    print(index_image)
    axs[index_count].imshow(image_3d[:,:,index_image])

### Change to the destination folder and save the 3D data as tiff file:

In [ ]:
os.getcwd()

In [ ]:
destination_path = root_path / 'dog_4_tiff'

In [ ]:
os.chdir(destination_path)

In [ ]:
os.getcwd()

In [ ]:
tifffile.imwrite('image.tiff', image_3d)

### Open napari and drag the tiff image into it to view the 3D structure:

In [ ]:
import napari

In [ ]:
viewer = napari.Viewer()

### How to annotate with Napari and segment anything:

plugins -> segment -> anything 2d annotator
compute embeddings

prompt

prompt point

segment object

image_1slice, gray